*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
# Getting the data
header = ['type', 'txt']
train_df = pd.read_csv(train_file_path, sep='\t', names=header)
test_df = pd.read_csv(test_file_path, sep='\t', names=header)
train_df

In [ ]:
# Transform categorical data to numerical
train_df['type'] = pd.factorize(train_df['type'])[0]
test_df['type'] = pd.factorize(test_df['type'])[0]
test_df.head()

In [ ]:
# Getting the labels
train_labels = train_df.pop('type')
test_labels = test_df.pop('type')

In [ ]:
print(test_df['txt'][0])

In [ ]:
# Cleaning data function
import re
def cleaning(df):
  clean = []
  for i in range(len(df)):
    temporal = re.sub('[.,!? ]', ' ', df['txt'][i])
    temporal = re.sub('[^a-zA-Z] ', '', temporal)
    temporal = temporal.lower()
    clean.append(temporal)
  return clean

In [ ]:
# Clean the data
train = cleaning(train_df)
test = cleaning(test_df)

In [ ]:
test[0]

In [ ]:
# Using tokenizer to assign each word to a number
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train)

train_token = tokenizer.texts_to_sequences(train)
test_token = tokenizer.texts_to_sequences(test)

vocabulary_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index

In [ ]:
print(test_token[0])

In [ ]:
# Adding an ending pad to 100 words filled with 0
pad = 100

train_token = keras.preprocessing.sequence.pad_sequences(train_token, padding='post', maxlen=pad)
test_token = keras.preprocessing.sequence.pad_sequences(test_token, padding='post', maxlen=pad)

In [ ]:
print(test_token[0])

In [ ]:
# Creating the model
from keras import layers
model = tf.keras.Sequential([
    layers.Embedding(vocabulary_size, 64, input_length=pad),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.summary()

In [ ]:
# Compiling the model
model.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

In [ ]:
# Training the model
model.fit(
    train_token, train_labels,
    validation_data=(test_token, test_labels),
    epochs=10,
    verbose=2
)

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  df_text = pd.DataFrame([pred_text]).rename(columns={0:'txt'})
  clean_text = cleaning(df_text)
  token_text = tokenizer.texts_to_sequences(clean_text)
  pad_text = keras.preprocessing.sequence.pad_sequences(token_text, padding='post', maxlen=pad)
  predict_list = model.predict(pad_text)[0][:len(token_text[0])]
  predict = predict_list.mean()
  if predict < 0.4:
    prediction = ([predict, 'ham'])
  else:
    prediction = ([predict, 'spam'])
  return (prediction)

pred_text = "our new mobile video service is live. just install on your phone to start watching."

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
pred_text = "our new mobile video service is live. just install on your phone to start watching."

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
